# Uniview module for LIGO event GW170817

*Aaron Geller, 2018*

### Imports and function definitions

In [1]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [198]:
Template = """mesh
{  
    data center    ./modules/$folderName/center.raw #I get errors if I don't pass something to geometry??
    data GRB    ./modules/$folderName/GRB.raw

    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec2f pfit 1.38504943e-05 3.73532968e-01
            vec1f eventTime -0.1 | public | desc "event time " | widget slider | range -30 30
            vec1f NSrad 10 | public | desc "NS radius " | widget slider | range 0 100
            vec1f GWrad 500 | public | desc "GW plane size " | widget slider | range 0 5000
            vec1f kilonovaRad 1000 | public | desc "kilonova max radius " | widget slider | range 0 1e5
            vec1f kilonovaMaxT 0.1 | public | desc "time at max kilonova radius " | widget slider | range 0 1
            vec1f GRBrad 0.4 | public | desc "GRB point radius " | widget slider | range 0 2
            vec1f GRBspeed 50 | public | desc "GRB speed " | widget slider | range 0 1000
            vec1f GRBMaxT 1 | public | desc "GRB fade time" | widget slider | range 0 10

        }
    }
    ############# gravitation waves
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/GW.gs
                vertexShader ./modules/$folderName/kilonova.vs
                fragmentShader   ./modules/$folderName/GW.fs
                
                stateManagerVar __objectName__.eventTime  eventTime                
                stateManagerVar __objectName__.pfit  pfit
                stateManagerVar __objectName__.GWrad  GWrad

                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    ############# NS 1
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/NS.gs
                vertexShader ./modules/$folderName/kilonova.vs
                fragmentShader   ./modules/$folderName/NS.fs
                parameter1f starNum 1. 
                
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.pfit  pfit
                stateManagerVar __objectName__.NSrad  NSrad
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    ############# NS 2
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/NS.gs
                vertexShader ./modules/$folderName/kilonova.vs
                fragmentShader   ./modules/$folderName/NS.fs
                parameter1f starNum 2. 

                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.pfit  pfit
                stateManagerVar __objectName__.NSrad  NSrad
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    ############# NS 1 "corona"
    ############# NS 2 "corona"

    ############# GRB
    pass
    {
        useDataObject GRB
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/GRB.gs
                vertexShader ./modules/$folderName/kilonova.vs
                fragmentShader   ./modules/$folderName/GRB.fs
                
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.GRBrad  GRBrad
                stateManagerVar __objectName__.GRBspeed  GRBspeed
                stateManagerVar __objectName__.GRBMaxT  GRBMaxT
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    ############# kilonova
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/kilonova.gs
                vertexShader ./modules/$folderName/kilonova.vs
                fragmentShader   ./modules/$folderName/kilonova.fs
                
                stateManagerVar __objectName__.eventTime  eventTime
                stateManagerVar __objectName__.kilonovaRad  kilonovaRad
                stateManagerVar __objectName__.kilonovaMaxT  kilonovaMaxT
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }

}"""

### Kilonova class

In [199]:
class Kilonova():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("kilonova.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("kilonova.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("kilonova.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("NS.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("NS.fs",dataFolder)
#         uvmod.Utility.ensurerelativepathexsists("corona.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("GW.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("GW.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("GRB.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("GRB.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [200]:
model = Kilonova(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [201]:
model.object.setcameraradius(10)
model.object.setcoord("Earth")
model.object.setname("Kilonova")
model.object.setguiname("/KavliLecture/Larson/Kilonova")
model.object.settargetradius(20)
model.object.showatstartup(False)
model.cr = 10000
modinfo.setname("Kilonova")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Shane Larson<sup>2</sup><br />(1)Adler Planetarium,br />(2)Northwestern University")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for LIGO event GW170817")
#modinfo.setthumbnail("data/R0010133.JPG")
modinfo.setversion("1.0")

generator.generate("Kilonova",[],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

## Helper Functions for modifing code
*Reload Module and Shaders in Uniview*

In [172]:
uvmod.Utility.senduvcommand(model.object.name+".reload; system.reloadallshaders")

*Copy modified Shader files and reload*

In [42]:
from config import Settings


In [44]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")